1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
  
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서로 LLM에 질문과 같이 전달

In [ ]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install -U langchain langchain-core langchainhub
%pip install -U pinecone

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

## chunk_size = 청크 하나를 구성하는 최대 토큰 수
## chunk_overlap = 청크들이 겹치는 토큰 수
## 예를 들어, chunk_size=1500, chunk_overlap=200이면
## 각 청크는 최대 1500 토큰까지 포함할 수 있으며,
## 각 청크는 이전 청크와 200 토큰이 겹치게 됩니다.
## 이 설정은 문서의 연속성을 유지하면서도
## 청크 간의 연결성을 높이는 데 도움이 됩니다.
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500, 
  chunk_overlap=200
  )

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [11]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")


In [12]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
index_name = 'tax-index'
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

if index_name not in [i["name"] for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# ✅ 5️⃣ from_documents() 호출 시 pinecone_client 제거
database = PineconeVectorStore.from_documents(
    document_list,
    embedding=embedding,
    index_name=index_name,
)

In [13]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query = query, k=3)


In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [15]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

[User Question]
{query}
"""

In [16]:
ai_response = llm.invoke(prompt)
ai_response.content

'연봉 5천만 원인 직장인의 소득세를 계산하려면 몇 가지 추가 정보가 필요합니다. 아래는 일반적인 소득세 계산 과정을 설명한 것입니다.\n\n1. **근로소득 공제**: 연봉에서 일정 금액을 공제하여 과세표준을 계산합니다.\n   - 근로소득 공제는 소득 구간에 따라 다르게 적용됩니다. 예를 들어, 5천만 원의 경우 대략 1,100만 원 내외의 근로소득 공제가 적용될 수 있습니다.\n\n2. **과세표준 계산**: 연봉에서 근로소득 공제를 뺀 금액이 과세표준이 됩니다.\n   - 예시: 5,000만 원 - 1,100만 원 = 3,900만 원 (과세표준)\n\n3. **기본세율 적용**: 과세표준에 해당하는 세율을 적용하여 소득세를 계산합니다.\n   - 한국의 경우, 과세표준에 따른 세율은 점진적으로 증가합니다. 예를 들어, 과세표준 1,200만 원 이하에는 6%, 그 이상에는 15%, 24%, 35% 등으로 증가하는 단계적 세율이 적용됩니다.\n\n4. **세액 공제**: 세액 공제 항목(예: 기본 공제, 보험료 공제, 기타 세액 공제 등)을 적용하여 최종 소득세를 결정합니다.\n\n각 단계마다 개인별로 적용되는 공제 및 세율이 다를 수 있으므로, 소득세 계산을 위해서는 보다 구체적인 공제 항목과 개인의 금융 상황을 고려해야 합니다. 이러한 요소들을 종합적으로 반영하여 계산해야 정확한 소득세액이 결정됩니다. \n\n정확한 소득세 계산을 위해 국세청의 간이세액표를 사용하거나, 공인된 세무 전문가의 도움을 받는 것이 좋습니다.'

In [17]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

combine_docs_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(database.as_retriever(), combine_docs_chain)

rag_chain.invoke({"question": query, "context": some_retrieved_context})


ModuleNotFoundError: No module named 'langchain.chains'